In [1]:
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
headers = {
    'User-Agent':
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.134 Safari/537.36'
}

In [ ]:
all_data_list = []
all_page_list = []
for x in range(1,25):

    rq = requests.get(f"https://www.trustpilot.com/categories/financial_institution?page={x}",headers=headers)

    test_soup = BeautifulSoup(rq.content, 'lxml')
    one_card = test_soup.find_all('div', class_='styles_businessUnitCard__1-z5m')
    for got_in_page in one_card:
        for that_url in got_in_page.find_all('a', class_= 'styles_linkWrapper__3x9X7', href=True):
            i_got_url = "https://www.trustpilot.com"+ that_url['href']
            all_page_list.append(i_got_url)

for items in all_page_list:

    in_page_r = requests.get(items, headers= headers)
    in_page_soup=BeautifulSoup(in_page_r.content, 'lxml')

    # Service Name
    try:
        # thaT_div_name = in_page_soup.find_all('div', id='business-unit-title')
        name = in_page_soup.find(
            'span', class_='title_displayName__TtDDM').text.strip()
    except AttributeError:
        name = "Not Available"
        pass

    # Trust Score
    try:
        t_c = in_page_soup.find('div', class_='styles_rating__NPyeH')
        Trust_score = t_c.find('p', class_='typography_typography__QgicV').text.strip()
    except AttributeError:
        Trust_score = "Score not Available"
        pass

    # Total Reviews
    try:
        total_reviews = in_page_soup.find('span', class_='styles_text__W4hWi').text.strip()
    except AttributeError:
        Trust_score = "Score not Available"
        pass

    # Website Name
    website_name = items

    # Addresses
    try:
        address = str([li.get_text(' ').split("<li>")for li in in_page_soup.select('ul.typography_typography__QgicV')]).strip("[").strip("]").strip("'")
    except AttributeError:
        address = "Address not Found"
        pass


    all_data_dict = {
        'Name' : name,
        'Trust Score': Trust_score,
        'Total Reviews' : total_reviews,
        'Website Name' : website_name,
        'Location': address
    }

    all_data_list.append(all_data_dict)
    df = pd.DataFrame(all_data_list)

df.index = df.index + 1
df.to_excel('D:\Programming\Python Coding\DcodeTech Projects\Trust Pilot\Data\Financial_Institution.xlsx', index_label='Sr No.')
